# Analysis of California Hospital Inpatient Mortality Rates 2016-2019

INFO: 
* The dataset contains risk-adjusted mortality rates, quality ratings, and number of deaths and cases for 6 medical conditions treated (Acute Stroke, Acute Myocardial Infarction, Heart Failure, Gastrointestinal Hemorrhage, Hip Fracture and Pneumonia) and 5 procedures performed (Abdominal Aortic Aneurysm Repair, Carotid Endarterectomy, Esophageal Resection, Pancreatic Resection, Percutaneous Coronary Intervention) in California hospitals.
* Dataset can be found at: 
    - https://catalog.data.gov/dataset/california-hospital-inpatient-mortality-rates-and-quality-ratings/resource/54ac2acc-f085-4b2d-9bf4-8368da30836e
    - Click 'Download' button on the page to download the dataset 

QUESTIONS: 

* What are the overall mortality rates due to heart failure in each county between 2016-2019?
    - Notes: Analysis of mortality rate for all cases of heart failure in the same county
* Using data from the previous question, is there a discernable yearly trend/pattern in the mortality rates of the county with the highest overall mortality rate due to heart failure?
    - Notes: Looks at heart failure mortality rates for each year of the county with the highest overall rate

In [ ]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
#Opens csv file and obtains data
with open('2016-2019_CA_Hos_Inp_Mortality_Rates_and_Qual_Ratings.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row for row in reader]
    csvfile.close()
# print (len(data))
data[0]

In [ ]:
#Converts a string to an int 
def toInt(str):
    if str.isdigit():
        return int(str)
    else:
        return 0

In [ ]:
#Converts a string to a float
def toFloat(str):
    return float(str)
f = toFloat(223)

In [ ]:
#Creates a list of data containing total # of Heart Failure related cases for desired county 
def totalHFCasesByCounty(county):
    cData = [county]
    # cData = []
    for d in data:
        if d['COUNTY'] == county and d['Procedure/Condition'] == 'Heart Failure':
            cData += [d['# of Cases']]
    return cData 

In [ ]:
#Creates a list of data containing total # of Heart Failure related deaths for desired county 
def totalHFDeathsByCounty(county):
    cData = [county]
    # cData = []
    for d in data:
        if d['COUNTY'] == county and d['Procedure/Condition'] == 'Heart Failure':
            cData += [d['# of Deaths']]
    return cData 

In [ ]:
#Calculates Mortality Rate from a list of data
def MortRateCalc(deaths, cases):
    if cases == 0:
        return 'N/A - No Cases Reported'
    return float("{:.2f}".format(float(deaths/cases) * 100))

# Question 1

# Matplot Vers.

In [ ]:
#Creates a list of counties, with each only listed once
counties = {d['COUNTY'] for d in data}
cList = list(counties)
cList.sort()

#Creates list containing info on heart failure cases and deaths by county and sorts them (alphabetical order by county)
cHFCases = []
cHFDeaths = []
for count in counties:
    cHFCases += [totalHFCasesByCounty(count)]
    cHFDeaths += [totalHFDeathsByCounty(count)]
cHFCases.sort()
cHFDeaths.sort()
# cHFCases
# cHFDeaths


In [ ]:
#Creates a list of totals for heart failure cases for each county (all cases for all hospitals for each county from all years)
cTotalCases = []
for cases in cHFCases:
    sum = 0
    for num in cases:
        sum += toInt(num)
    cTotalCases += [sum]
# cTotalCases

#Creates a list of totals for heart failure deaths for each county (all deaths for all hospitals for each county from all years)
cTotalDeaths = []
for cases in cHFDeaths:
    sum = 0
    for num in cases:
        sum += toInt(num)
    cTotalDeaths += [sum]
# cTotalDeaths


In [ ]:
#Creates a list of calculated mortality rates (all counties from all years)
cMortRate = []
for i in range(0, len(cTotalCases)):
    cMortRate.append( toFloat(MortRateCalc(cTotalDeaths[i], cTotalCases[i])))
# cMortRate


In [ ]:
#Creates scatter plot based on overall County Heart Failure cases and deaths
plt.figure(figsize=(30,10))
plt.title('Heart Failure Related Cases by County', fontdict=None, loc='center', pad=None)
plt.xlabel('Total Heart Failure Cases')
plt.ylabel('Total Heart Failure Deaths')
plt.scatter(cTotalCases, cTotalDeaths)
plt.show()

In [ ]:
#Creates bar graph of Overall Heart Failure Mortality Rates for all Counties (0-10% as they don't exceed 10%)
plt.figure(figsize=(20,10), dpi = 100)
plt.title('Heart Failure Mortality Rates by County')
plt.ylim([0, 10])
plt.xlabel('County')
plt.xticks(rotation=90)
plt.ylabel('Mortality Rate (%)')
plt.bar(cList, cMortRate)
plt.show()

# Pandas/Numpy Vers. 

In [ ]:
#Read data file; Get unique counties and years from dataframe
df = pd.read_csv('2016-2019_CA_Hos_Inp_Mortality_Rates_and_Qual_Ratings.csv')
pCounties = df['COUNTY'].unique()

#Hides non-heart failure conditions 
df = df[df['Procedure/Condition'] == 'Heart Failure']
# df


In [ ]:
#Cleans data (converts values/holders so that 0s are read instead of periods)
removePeriodsCases = df['# of Cases']
removedListCases = []
for p in removePeriodsCases:
    if isinstance(p, float) or not p.isdigit():
        p = '0'
    removedListCases.append(p)

removePeriodsDeaths = df['# of Deaths']
removedListDeaths = []
for p in removePeriodsDeaths:
    if isinstance(p, float) or not p.isdigit():
        p = '0'
    removedListDeaths.append(p)


In [ ]:
#Deletes original columns and replaces them w/ copies containing cleaned data for better reading; Converts info from string to int types
del df['# of Cases'] 
df.insert(7,'# of Cases',removedListCases)
pd.set_option('display.max_rows', None)
# df.reset_index(drop=True, inplace=True)
df

del df['# of Deaths'] 
df.insert(6,'# of Deaths',removedListDeaths)
pd.set_option('display.max_rows', None)
df.reset_index(drop=True, inplace=True)
df

df['# of Cases'] = df['# of Cases'].astype(int)
df['# of Deaths'] = df['# of Deaths'].astype(int)

df

In [ ]:
#Generates pivot table w/ final row/column consisiting of totals (Cases, Deaths) 
df1 = pd.pivot_table(data=df, values=['# of Cases','# of Deaths'], index=['COUNTY'], columns='YEAR',aggfunc=np.sum, fill_value=0, margins=True, margins_name='Total')
df1.columns
df1

#Gets 'Total' columns for deaths and cases
cbyCount = df1['# of Cases', 'Total']
dbyCount = df1['# of Deaths', 'Total']

#Creates a list of calculated Mortality Rates for each county (all years combined) and inserts list as a new column
mortRate = []
for ind in range(0, len(cbyCount)):
    cDeaths = dbyCount[ind]
    cCases = cbyCount[ind]
    mr = MortRateCalc(cDeaths, cCases)
    mortRate.append(mr)
mortRate
len(df1.columns)
df1.insert(10, 'Mortality Rate (%)', mortRate)
df1


In [ ]:
#Creates a scatter plot showing number of deaths by cases for each county from 2016-2019 combined
df1.plot.scatter(x='# of Cases', y='# of Deaths', s = 10, figsize=(10,10))
plt.grid() 
plt.title('Deaths by Cases')
plt.xlabel('Cases')
plt.ylabel('Deaths')
plt.show()


In [ ]:
#Creates a bar graph consisting of mortality rates for each county 
df1.plot.bar( y='Mortality Rate (%)', figsize=(18, 10))
plt.show()


In [ ]:
df1.describe()

In [ ]:
list(df1.columns)

In [ ]:
list(df1.index)

In [ ]:
#Gets Top 5 Counties with highest overall mortality rates 
df1 = df1.sort_values(['Mortality Rate (%)'], ascending=False).head(5)
df1

# Question 2

In [ ]:
#Creates a pivot table with yearly information for the county with the highest overall mortality rate (Lassen)
df2 = df[df['COUNTY'] == 'Lassen']
df2 = pd.pivot_table(data=df2, values=['# of Cases','# of Deaths'], index=['YEAR'])

#Calculates the mortality rate for each year; adds as a column to pivot table
yrDeaths = df2['# of Deaths']
yrCases = df2['# of Cases']
yrlyMortRate = []

for ind in range(0, len(yrCases)):
    yrlyMortRate.append(MortRateCalc(df2['# of Deaths'].iloc[ind], df2['# of Cases'].iloc[ind]))
    
df2.insert(2, 'Mortality Rate (%)', yrlyMortRate)
df2


In [ ]:
#Plots a graph showing mortality rates the county with highest overall mortality rate (Lassen) for each year (2016-2019)
plt.figure(figsize=(10, 10))
x = np.linspace(2016, 2019, 4)
plt.plot(x, df2['Mortality Rate (%)'], label='Mortality Rate 2016-2019')
plt.xticks(x)
plt.xlabel('Year')
plt.ylabel('Mortality Rate (%)')
plt.legend()
plt.show()

# Discussion

The analysis of the data provided by the csv file indicates that Lassen County had the highest overall mortality rate for Heart Failure cases between 2016 and 2019. The overall mortality rate between those years was 8.41%. Closer inspection of the data provided shows that there has been an increasing trend in the Heart Failure mortality rate in Lassen County for each year. While there is a slight dip from 2017 to 2018, the overall mortality rate rose from 0% in 2016 to approximately 16.67% in 2019.